### Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

# Neural Network Model

In [ ]:
class language_model(nn.Module):
    def __init__(self,context,embedding_size,hidden_size,number_of_layers,vocab,learning_rate):
        super(language_model,self).__init__()

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.context = context
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.number_of_layers = number_of_layers
        self.vocab = vocab

        layers = []
        for i in range(number_of_layers):
            layers.append(nn.Linear(hidden_size,hidden_size))

        self.model = nn.Sequential(
            nn.Embedding(vocab,embedding_size),
            nn.Linear(vocab*context,hidden_size),
            layers,
            nn.Linear(hidden_size,vocab),
            nn.LogSoftmax(dim=1)
        ).to(self.device)

        self.loss_function = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.model.parameters(), lr=learning_rate)
    
    def forward(self,input_data):
        return self.model(input_data)


    def train(self,ngrams,word_to_id,num_epochs):
        self.model.train()
        for epoch in range(num_epochs):
            total_loss = 0
            for context_words, current_word in ngrams:

                context_word_ids = torch.tensor([word_to_id[word] for word in context_words], dtype=torch.long).to(self.device)

                self.model.zero_grad()

                log_probs = self.model(context_word_ids)
               
                loss = self.loss_function(log_probs, torch.tensor([word_to_id[current_word]], dtype=torch.long).to(self.device))

                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()
            losses.append(total_loss)
            print("Loss after Epoch",epoch,":",total_loss)
        print(losses)


    def test(self,ngrams,word_to_id):
        self.model.eval()
        with torch.no_grad():
            num_correct = 0
            total=0
            for context_words, current_word in ngrams:

                    context_word_ids = torch.tensor([word_to_id[word] for word in context_words], dtype=torch.long).to(self.device)

                    log_probs = self.model(context_word_ids)

                    word_index = torch.argmax(log_probs)

                    if word_to_id[current_word] == word_index:
                        num_correct+=1

                    total+=1

                    loss = self.loss_function(log_probs, torch.tensor([word_to_id[current_word]], dtype=torch.long).to(self.device))

                    total_loss += loss.item()
            
            print("Total Loss:",total_loss)
            print("Accuracy:",num_correct/total_loss)



    



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=aa6c7eb5-d731-4a9a-8f73-5821cfc2b6d5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>